<a href="https://colab.research.google.com/github/sberniz/DS-Unit-4-Sprint-1-NLP/blob/main/module3-document-classification/LS_DS_413_Document_Classification_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [2]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [4]:
target = train['ratingCategory']
data = train.drop(columns=['id','ratingCategory'])

In [5]:
data

,description
0,"\nSometimes, when whisky is batched, a few lef..."
1,\nAn uncommon exclusive bottling of a 6 year o...
2,\nThis release is a port version of Amrut’s In...
3,\nThis 41 year old single cask was aged in a s...
4,"\nQuite herbal on the nose, with aromas of dri..."
...,...
4082,\nWhat lies beneath the surface of Dewar’s? He...
4083,\nAfter 6 to 7 years of maturation in bourbon ...
4084,"\nBright, delicate, and approachable. While no..."
4085,\nI’m calling this the pitmaster’s dram: the n...


In [6]:
target

0       1
1       0
2       1
3       1
4       1
       ..
4082    1
4083    1
4084    1
4085    1
4086    1
Name: ratingCategory, Length: 4087, dtype: int64

### Define Pipeline Components

In [7]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [8]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'clf__max_depth':(5,10)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(train['description'],train['ratingCategory'])

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   40.1s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [9]:
grid_search.best_score_

0.7134821473995445

In [10]:
grid_search.predict(['wiskey i love it ', 'you won the lottery in Nigeria'])

array([1, 1])

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [11]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [12]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [13]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [14]:
subNumber = 0

In [15]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [16]:
from sklearn.decomposition import TruncatedSVD

In [17]:
svd = TruncatedSVD(n_components=100,
                   algorithm='randomized',
                   n_iter=5)

In [18]:
lsi = Pipeline([('vect', vect), ('svd', svd)])
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier(n_jobs=-1,random_state=42)

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

In [19]:
print(pipe)

Pipeline(memory=None,
         steps=[('lsi',
                 Pipeline(memory=None,
                          steps=[('vect',
                                  TfidfVectorizer(analyzer='word', binary=False,
                                                  decode_error='strict',
                                                  dtype=<class 'numpy.float64'>,
                                                  encoding='utf-8',
                                                  input='content',
                                                  lowercase=True, max_df=1.0,
                                                  max_features=None, min_df=1,
                                                  ngram_range=(1, 2), norm='l2',
                                                  preprocessor=None,
                                                  smooth_idf=True,
                                                  stop_words='english',
                                                  strip_a

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [20]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=7,n_jobs=-1, verbose=10)
grid_search.fit(data['description'], target)

Fitting 7 folds for each of 24 candidates, totalling 168 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   16.0s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   22.3s
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 

GridSearchCV(cv=7, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [22]:
grid_search.best_score_

0.723515757956182

### Make a Submission File

In [23]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [24]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [27]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [28]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [73]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [31]:
# Continue Word Embedding Work Here
import spacy
nlp = spacy.load("en_core_web_lg")

In [33]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [34]:
X = get_word_vectors(data['description'])
len(X) == len(data['description'])

True

In [36]:
clf1 = RandomForestClassifier()


In [37]:
clf1.fit(X,target)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [38]:
clf1.score(X,target)

1.0

In [48]:
from xgboost import XGBClassifier


In [101]:
import numpy as np

In [54]:
clf2 = XGBClassifier(random_state=42)

In [102]:
clf2.fit(np.array(X),target)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

0.857841937851725

In [87]:
param_dist = {
    
    'max_depth' :[5,10,20],
    'n_estimators':[100,110,120,150,200]
}

In [113]:
grid_search2 = GridSearchCV(clf2,param_dist,cv=5,n_jobs=-1,verbose=10)
grid_search2.fit(np.array(X),target)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   50.5s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 11.3min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed: 16.1min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 23.5min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 31.7min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed: 42.5min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed: 53.6min
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed: 62.0min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='multi:softprob',
                                     random_state=42, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': [5, 10, 20],
                         'n_estimators': [100, 110, 120, 150, 200]},
             pre_dispatch='2*n_jobs', refit=True, return_tr

In [114]:
grid_search2.best_params_

{'max_depth': 5, 'n_estimators': 120}

In [115]:
grid_search2.best_score_

0.7430871487013434

In [117]:
ypred4 = grid_search2.predict(X_test)

In [123]:
from sklearn.tree import DecisionTreeClassifier

In [124]:
clf3 = DecisionTreeClassifier()

In [125]:
clf3.fit(X,target)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [126]:
clf3.score(X,target)

1.0

In [127]:
ypred5 = clf3.predict(X_test)

### Make a Submission File

In [40]:
# Predictions on test sample
X_test = get_word_vectors(test['description'])
pred = clf1.predict(X_test)

In [95]:
ypred2 = grid_search2.predict(X_test)

In [104]:
ypred3 = clf2.predict(X_test)

In [106]:
ypred3

array([1, 1, 1, ..., 1, 1, 1])

In [128]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':ypred5})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [130]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [109]:
from sklearn.metrics import accuracy_score

In [111]:
ypredt = clf2.predict(X)
accuracy_score(target,ypredt)

0.857841937851725

In [131]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 

    ```Sentiment analysis (also known as opinion mining or emotion AI) refers to the use of natural language processing, use of data science to systematically identify, extract, quantify, and study affective states and subjective information. for example detect the of the person writing a statemnet. such as happy/sad,angry, etc.  ```
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    ```Yes, Document Clasification can be used to classified a document(Text data) into different categories, such as technology, physics, etc. and while it can also classify a sentiment.. a sentiment analysis just analyses the sentiment or emotion of the sentence. ```
    - How do create labeled sentiment data? Are those labels really sentiment? 
    ```We need define clear rules, for example is profanity is labled as negative, we should keep consistent and test randomness,the labels are not really sentiments ```
    - What are common applications of sentiment analysis?
    ```Nowadays probably most common application is for targeted marketing, which analyses person state from for example reviews, to market certain products```
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
```In my case it workedthe same on both on the training data. however,one reason is because the sentiment was different```
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?